# 0. Import packages

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

ModuleNotFoundError: No module named 'bs4'

# 1. Create function to web scrap data from url

In [103]:
def extract_data(keyword, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    
    items = soup.select('div.lister-item.mode-detail')
    
    movies_data = pd.DataFrame(columns=['title', 'rating', 'categories', 'description'])
    
    titles = []
    ratings = []
    descriptions = []
    categories = []
    for i in range(len(items)):
        tit = soup.select('h3.lister-item-header a')[i].get_text()
        try: 
            rat = str(items[i].select('div.inline-block.ratings-imdb-rating strong')).split('>')[1].split('<')[0]
        except IndexError:
            rat = ''
        desc = str(items[i].select('p[class=""]'))[14:-5]
        elems = str(items[i].select('span[class="genre"]'))[22:-8].replace(',', '').split(' ')
        cat = [x for x in elems if x!='']
    
        titles.append(tit)
        ratings.append(rat)
        descriptions.append(desc)
        categories.append(cat)
        
    movies_data['title'] = titles
    movies_data['rating'] = ratings
    movies_data['categories'] = categories
    movies_data['description']  = descriptions    
    movies_data['keyword'] = keyword
    
    return movies_data

# 2. Read link with keywords

In [104]:
# Read master link containing keywords
master_url = 'https://www.imdb.com/search/keyword/?sort=moviemeter,asc&mode=detail&page=1&ref_=kw_ref_key'
master_response = requests.get(master_url)
master_soup = BeautifulSoup(master_response.text)

# Divide by rows
rows = master_soup.select('div.table-row')

# Obtain keywords
keywords = []
for i in range(len(rows)):
    word = master_soup.select('div.table-row a')[i].get_text()[1:-1]
    keywords.append(word)
    
# Read link
links = [a.attrs.get('href') for a in master_soup.select('div.table-row  a')]

# Convert link to url
urls = ['https://www.imdb.com'+i for i in links]

# Create DataFrame with keywords and urls
keywords_links = pd.DataFrame(columns=['keywords', 'url'])
keywords_links['keywords'] = keywords
keywords_links['url'] = urls

# 3. Scrape data for each of the keywords

In [105]:
# Get data for all keywords
master_df = pd.DataFrame([])

for i in range(len(keywords_links)):
    df = extract_data(keywords_links.iloc[i].keywords, keywords_links.iloc[i].url)
    master_df = pd.concat([master_df, df])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


In [106]:
# Master DataFrame with all movies data
master_df

,title,rating,categories,description,keyword
0,The Batman,,"[Action, Crime, Drama]","In his second year of fighting crime, Batman u...",action hero
1,Sin tiempo para morir,7.6,"[Action, Adventure, Thriller]",James Bond has left active service. His peace ...,action hero
2,Venom: Carnage liberado,6.3,"[Action, Adventure, Sci-Fi]",Eddie Brock attempts to reignite his career by...,action hero
3,Uncharted,,"[Action, Adventure]","The story is a prequel to the games, starring ...",action hero
4,Shang-chi y la leyenda de los diez anillos,7.9,"[Action, Adventure, Fantasy]","Shang-Chi, the master of weaponry-based Kung F...",action hero
...,...,...,...,...,...
45,Army of Thieves,6.3,"[Action, Comedy, Crime]","A prequel, set before the events of Army of th...",zombie
46,Posesión infernal,6.5,[Horror],"Five friends head to a remote cabin, where the...",zombie
47,ParaNorman,7.0,"[Animation, Adventure, Comedy]","A misunderstood boy takes on ghosts, zombies a...",zombie
48,Evil Dead II,7.7,"[Comedy, Horror]",The lone survivor of an onslaught of flesh-pos...,zombie


In [107]:
# Save data 
master_df.to_csv('/Users/rafaelmateus/Desktop/movies.csv')